In [1]:
%run bert.ipynb

attention pattern raw MATCH!!!!!!!!
 SHAPE (2, 12, 3, 3) MEAN: 0.004686 STD: 0.1078 VALS [0.0596 -0.04372 -0.1686 0.1675 -0.05231 0.06658 0.2362 0.162 0.07567 -0.15...]
attention MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: -0.001142 STD: 0.1204 VALS [-0.1816 -0.05433 -0.2183 -0.009458 0.002117 -0.04871 -0.2099 -0.01656 -0.1198 0.1005...]
bert MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: -0.001554 STD: 0.1736 VALS [-0.08316 -0.09165 -0.03188 -0.03013 0.1001 0.09549 -0.1046 0.07742 0.0424 0.05553...]
bert mlp MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: -0.0001934 STD: 0.1044 VALS [-0.1153 0.1189 -0.0813 0.1021 0.0296 0.06182 0.0341 0.1446 0.2622 -0.08507...]
layer norm MATCH!!!!!!!!
 SHAPE (20, 10) MEAN: -1.431e-08 STD: 1.003 VALS [0.6906 -0.84 1.881 1.711 -0.5116 -0.9577 -0.1387 -0.6943 -0.6741 -0.4662...]
bert MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: -5.381e-09 STD: 1 VALS [0.007132 -0.04372 0.6502 -0.5972 -1.097 0.7267 0.1275 -0.6035 -0.2226 0.2145...]
{'input_ids': [[101, 8667, 117, 146, 1821, 170, 56

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
import transformers
from tqdm.notebook import tqdm
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")

In [3]:
def make_bert():
    my_bert = Bert(
        vocab_size=28996, hidden_size=384, max_position_embeddings=512, 
        type_vocab_size=2, dropout=0.1, intermediate_size=1536, 
        num_heads=6, num_layers=2, num_classes=2
    )
    return my_bert

In [67]:
import torchtext
import random
from torch.utils.data import DataLoader
data_train, data_valid, data_test = torchtext.datasets.WikiText2(root='.data', split=('train', 'valid', 'test'))

data_train_list = list(data_train)

train_dataloader = DataLoader(data_train_list, batch_size=16, shuffle=True)
test_dataloader = DataLoader(data_test, batch_size=16)
valid_dataloader = DataLoader(data_test, batch_size=16)

In [64]:
bert = make_bert().cuda()
lossfn = nn.CrossEntropyLoss()

In [65]:
lr = 1e-4
optimizer = torch.optim.Adam(bert.parameters(), lr)

In [77]:
def train(model, optimizer, data, lossfn, epochs=1, max_seq_len=512):
    model.train()
    i = 0
    for epoch in range(epochs):
        for X in tqdm(data):
            optimizer.zero_grad()
            
            # token processing
            tokens = tokenizer(X, padding='longest', max_length=max_seq_len, truncation=True)
            # the original input
            unmasked_tokens = torch.tensor(tokens.input_ids, dtype=torch.long).cuda()
            zero_tokens = unmasked_tokens == 0
            rand_nums = torch.rand(unmasked_tokens.shape).cuda() <= 0.15

            masked_tokens = unmasked_tokens.clone()
            masked_tokens[rand_nums] = tokenizer.mask_token_id
            masked_tokens[zero_tokens] = 0

            output, _classifications = model(masked_tokens)
            masked_mask = masked_tokens == tokenizer.mask_token_id
            #expected_output_at_masks = torch.masked_select(unmasked_tokens,    masked_mask)
            expected_output_at_masks = unmasked_tokens[masked_mask]
            unnormed_probs_at_masks = output[masked_mask]
            loss = lossfn(unnormed_probs_at_masks, expected_output_at_masks) 

            
            #print(unmasked_tokens)
            #print(masked_tokens)
            #print(output)
            #print(expected_output_at_masks)
            #print(unnormed_probs_at_masks)
            #break
            
            loss.backward()
            optimizer.step()
            i += 1
            if i % 100 == 0:
                print(loss)


train(bert, optimizer, train_dataloader, lossfn, epochs=10)

  0%|          | 0/2295 [00:00<?, ?it/s]

tensor(6.9441, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.6596, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.8445, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.2742, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.5050, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.7823, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(7.1786, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.8116, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.7298, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(7.2527, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.7060, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(7.3016, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9887, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.8682, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.8267, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.1301, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.7391, device='cuda:0', grad_fn=

  0%|          | 0/2295 [00:00<?, ?it/s]

tensor(7.0288, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.6335, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.9011, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.4277, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.8137, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.3405, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.8064, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.1017, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.7839, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.6160, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.6577, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.3534, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.3491, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.2084, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(7.0864, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.3401, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.4482, device='cuda:0', grad_fn=

  0%|          | 0/2295 [00:00<?, ?it/s]

...

tensor(5.7731, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.0478, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.0631, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.2425, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.1059, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.0684, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.1403, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.6704, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.0470, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.2939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.2296, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8174, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8152, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.2242, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.5457, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.4145, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.2403, device='cuda:0', grad_fn=

  0%|          | 0/2295 [00:00<?, ?it/s]

tensor(5.6757, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.3285, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.7491, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.2499, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.6170, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.1096, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.1254, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.1402, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9036, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.0319, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.3096, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.1381, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.2978, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.7501, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.5633, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.2182, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.2490, device='cuda:0', grad_fn=

  0%|          | 0/2295 [00:00<?, ?it/s]

tensor(5.8705, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9807, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9492, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.4687, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.0838, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9032, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.1318, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.4011, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.0984, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.7681, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.2203, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8081, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.1567, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.0383, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8104, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.6742, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8322, device='cuda:0', grad_fn=

In [73]:
def ascii_art_probs(model, sentence):
    token_ids = torch.Tensor(tokenizer.encode(sentence)).long().unsqueeze(0)
    mask_idxs = set()
    for idx, token_id in enumerate(token_ids[0]):
        if token_id == 103: # 103 == [MASK]
            mask_idxs.add(idx)
    unnormalized_output, _classifications = bert(token_ids.cuda())
    output = torch.log_softmax(unnormalized_output, dim=-1)
    top_k = torch.topk(output, 5, dim=-1)
    results = []
    for seq_i, seq_top_k in enumerate(top_k.indices[0]):
        if seq_i in mask_idxs:
            results.append(list(zip(tokenizer.convert_ids_to_tokens(seq_top_k), [round(x.item(), 2) for x in top_k.values[0][seq_i].exp()])))
    for l in results:
        print(l)

ascii_art_probs(bert, "The [MASK] loves to eat.")

[('the', 0.09), (',', 0.02), ('of', 0.02), ('to', 0.01), ('was', 0.01)]
